In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import collections

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
class_names = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']

In [3]:
from torchvision import datasets, transforms
data_path = '/data-unversioned/p1ch6/'
cifar10 = datasets.CIFAR10(
    data_path, train=True, download=True,
    transform=transforms.Compose([
        transforms.RandomCrop(size=[32,32], padding=4),
        transforms.RandomHorizontalFlip(),                       
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))


Extracting /data-unversioned/p1ch6/cifar-10-python.tar.gz to /data-unversioned/p1ch6/


In [4]:
cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

Files already downloaded and verified


In [5]:
class resblock(nn.Module):
    def __init__(self, n):
        super(resblock, self).__init__()
        self.conv = nn.Conv2d(n, n, kernel_size=3,
                              padding=1, bias=False)  
        self.batch_norm = nn.BatchNorm2d(num_features=n)

        torch.nn.init.kaiming_normal_(self.conv.weight,
                                      nonlinearity='relu')  
        torch.nn.init.constant_(self.batch_norm.weight, 0.5)
        torch.nn.init.zeros_(self.batch_norm.bias)

    def forward(self, x):
        out = self.conv(x)
        out = self.batch_norm(out)
        out = torch.relu(out)
        return out + x

In [6]:
class deep(nn.Module):
    def __init__(self, n1=32, n_blocks=10):
        super().__init__()
        self.n1 = n1
        self.conv1 = nn.Conv2d(3, n1, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(n1, 2*n1, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(2*n1, 4*n1, kernel_size=3, padding=1)
        self.resblocks = nn.Sequential(
            *(n_blocks * [resblock(n=n1)]))
        self.resblocks2 = nn.Sequential(
            *(n_blocks * [resblock(n=n1*2)]))
        self.resblocks3 = nn.Sequential(
            *(n_blocks * [resblock(n=n1*4)]))
        self.fc1 = nn.Linear(8 * 8 * n1*4, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 10)
        
    def forward(self, x):
        out = torch.relu(self.conv1(x))
        out = self.resblocks(out)
        out = F.max_pool2d(out, 2) 
        out = self.conv2(out)
        out = self.resblocks2(out)
        out = F.max_pool2d(out, 2) 
        out = self.conv3(out)
        out = self.resblocks3(out)
        out = out.view(-1, 8 * 8 * self.n1*4)
        out = torch.relu(self.fc1(out))
        out = torch.relu(self.fc2(out))
        out = self.fc3(out)
        return out

In [7]:
device = (torch.device('cuda') if torch.cuda.is_available()
          else torch.device('cpu'))
print(f"Training on device {device}.")

Training on device cuda.


In [8]:
import datetime

def training_loop(n_epochs, optimizer, model, loss_fn, train_loader, schedular):
    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        for imgs, labels in train_loader:
            imgs = imgs.to(device=device)  # <1>
            labels = labels.to(device=device)
            outputs = model(imgs)
            loss = loss_fn(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loss_train += loss.item()

        schedular.step() 
        print('{} Epoch {}, Training loss {}'.format(
            datetime.datetime.now(), epoch,
            loss_train / len(train_loader)))

In [9]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64,
                                           shuffle=True)

model =  deep().to(device=device)# <1>
optimizer = optim.SGD(model.parameters(), lr=1e-2, weight_decay=0.01)
schedular = optim.lr_scheduler.MultiStepLR(optimizer, [50,75], gamma=0.1)
loss_fn = nn.CrossEntropyLoss()

In [10]:
from torchsummary import summary
summary(model,(3,32,32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 32, 32]             896
            Conv2d-2           [-1, 32, 32, 32]           9,216
       BatchNorm2d-3           [-1, 32, 32, 32]              64
          resblock-4           [-1, 32, 32, 32]               0
            Conv2d-5           [-1, 32, 32, 32]           9,216
       BatchNorm2d-6           [-1, 32, 32, 32]              64
          resblock-7           [-1, 32, 32, 32]               0
            Conv2d-8           [-1, 32, 32, 32]           9,216
       BatchNorm2d-9           [-1, 32, 32, 32]              64
         resblock-10           [-1, 32, 32, 32]               0
           Conv2d-11           [-1, 32, 32, 32]           9,216
      BatchNorm2d-12           [-1, 32, 32, 32]              64
         resblock-13           [-1, 32, 32, 32]               0
           Conv2d-14           [-1, 32,

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [11]:
training_loop(  
    n_epochs = 100,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader, 
    schedular = schedular
)

2021-08-01 18:08:05.286146 Epoch 1, Training loss 1.8512785703020023
2021-08-01 18:08:59.406138 Epoch 2, Training loss 1.3609966569391967
2021-08-01 18:09:53.595807 Epoch 3, Training loss 1.1326002294145276
2021-08-01 18:10:48.778205 Epoch 4, Training loss 1.0084592962204038
2021-08-01 18:11:42.893644 Epoch 5, Training loss 0.9158232118315099
2021-08-01 18:12:36.629441 Epoch 6, Training loss 0.8398586007792627
2021-08-01 18:13:30.930587 Epoch 7, Training loss 0.7930407429214024
2021-08-01 18:14:25.813958 Epoch 8, Training loss 0.7468117957987139
2021-08-01 18:15:20.242801 Epoch 9, Training loss 0.7126480972065645
2021-08-01 18:16:14.565080 Epoch 10, Training loss 0.6732981191266834
2021-08-01 18:17:09.083309 Epoch 11, Training loss 0.6482543961699966
2021-08-01 18:18:03.455659 Epoch 12, Training loss 0.6245960493755462
2021-08-01 18:18:57.937951 Epoch 13, Training loss 0.6039635857657704
2021-08-01 18:19:52.303475 Epoch 14, Training loss 0.5864547297472844
2021-08-01 18:20:46.671516 Ep

In [12]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64,
                                           shuffle=False)
val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64,
                                         shuffle=False)

def validate(model, train_loader, val_loader):
    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correct = 0
        total = 0

        with torch.no_grad():  # <1>
            for imgs, labels in loader:
                imgs = imgs.to(device=device)
                labels = labels.to(device=device)
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1) # <2>
                total += labels.shape[0]  # <3>
                correct += int((predicted == labels).sum())  # <4>

        print("Accuracy {}: {:.5f}".format(name , correct / total))

validate(model, train_loader, val_loader)

Accuracy train: 0.96502
Accuracy val: 0.90160


In [13]:
optimizer = optim.SGD(model.parameters(), lr=0.0001, weight_decay=0.01)

In [14]:
training_loop(  
    n_epochs = 10,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader, 
    schedular = schedular
)

2021-08-01 19:38:42.768389 Epoch 1, Training loss 0.10627329102038499
2021-08-01 19:39:36.524698 Epoch 2, Training loss 0.10763003118336201
2021-08-01 19:40:30.553736 Epoch 3, Training loss 0.10603393785312505
2021-08-01 19:41:24.669551 Epoch 4, Training loss 0.10708468248996207
2021-08-01 19:42:18.637775 Epoch 5, Training loss 0.1031107565964503
2021-08-01 19:43:12.655476 Epoch 6, Training loss 0.10261108724833906
2021-08-01 19:44:07.072960 Epoch 7, Training loss 0.10253930071611768
2021-08-01 19:45:01.214629 Epoch 8, Training loss 0.10153764712355097
2021-08-01 19:45:55.015156 Epoch 9, Training loss 0.10367591272744224
2021-08-01 19:46:48.864207 Epoch 10, Training loss 0.101511084739014


In [15]:
validate(model, train_loader, val_loader)

Accuracy train: 0.96718
Accuracy val: 0.90100
